# Visitor-Exhibitor Matching System

Goal: Match visitors to relevant exhibitors based on their answers and exhibitor categories

In [8]:
import pandas as pd
import numpy as np
import json
from fuzzywuzzy import process  # pip install fuzzywuzzy

## Step 1: Load the data files

Just loading all the CSVs we need

In [15]:
# Visitors data
visitors_df = pd.read_csv('visitors.csv')
answers_df = pd.read_csv('visitors_answers.csv')
questions_df = pd.read_csv('visitors_questions.csv')

# Exhibitors data  
exhibitors_df = pd.read_csv('exhibitors.csv')
exhibitor_categories_df = pd.read_csv('exhibitor_categories.csv')

print(f"Loaded {len(visitors_df)} visitors and {len(exhibitors_df)} exhibitors")
# lets see what we have
print(visitors_df.columns.tolist())
print(exhibitors_df.columns.tolist())

Loaded 100 visitors and 35 exhibitors
['email', 'gender', 'id', 'data']
['exhibitorid', 'Name', 'MainCategories']


In [33]:
# First, let's see what columns each dataframe actually has
print("\n=== Actual column names in each dataset ===")
print(f"Visitors columns: {visitors_df.columns.tolist()}")
print(f"Answers columns: {answers_df.columns.tolist()}")
print(f"Questions columns: {questions_df.columns.tolist()}")
print(f"Exhibitors columns: {exhibitors_df.columns.tolist()}")
print(f"Exhibitor Categories columns: {exhibitor_categories_df.columns.tolist()}")

# Create a function to analyze each dataset
def analyze_dataset(df, name):
    print(f"\n=== Analysis of {name} ===")
    
    # Basic info
    print(f"\nShape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    
    # Data types
    print(f"\nData types:")
    print(df.dtypes)
    
    # Sample data
    print(f"\nFirst 5 rows:")
    print(df.head())
    
    # Missing values
    print(f"\nMissing values:")
    print(df.isnull().sum())
    
    # For key columns, show unique value counts
    print(f"\nUnique value counts for key columns:")
    for col in df.columns:
        unique_count = df[col].nunique()
        if unique_count < 20:  # Show actual values if not too many
            print(f"  {col}: {unique_count} unique values")
            print(f"    {df[col].value_counts().head(10).to_dict()}\n")
        else:
            print(f"  {col}: {unique_count} unique values")
    
    # Numeric columns stats
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    if len(numeric_cols) > 0:
        print(f"\nNumeric column statistics:")
        print(df[numeric_cols].describe())
    
    # For any text columns, show max length
    text_cols = df.select_dtypes(include=['object']).columns
    if len(text_cols) > 0:
        print(f"\nText column lengths:")
        for col in text_cols:
            if col in df.columns:
                try:
                    max_len = df[col].astype(str).str.len().max()
                    min_len = df[col].astype(str).str.len().min()
                    avg_len = df[col].astype(str).str.len().mean()
                    print(f"  {col}: min={min_len}, max={max_len}, avg={avg_len:.1f}")
                except Exception as e:
                    print(f"  {col}: Unable to calculate lengths - {e}")
    
    print("\n" + "="*50)

# Show the JSON data structure for visitors
def show_json_structure(visitors_df, num_samples=3):
    print("\n=== Sample JSON data structure from visitors' data field ===")
    for i, row in visitors_df.head(num_samples).iterrows():
        print(f"\nVisitor ID: {row['id']}")
        try:
            data = json.loads(row['data'])
            print(json.dumps(data[:2], indent=2))  # Show first 2 entries for each visitor
            print(f"... (Total {len(data)} entries)")
        except Exception as e:
            print(f"Error parsing JSON: {e}")
    print("\n" + "="*50)

# Analyze each dataset
analyze_dataset(visitors_df, "Visitors")
analyze_dataset(answers_df, "Answers")
analyze_dataset(questions_df, "Questions")
analyze_dataset(exhibitors_df, "Exhibitors")
analyze_dataset(exhibitor_categories_df, "Exhibitor Categories")

# Show sample JSON structure
show_json_structure(visitors_df)

# Also check the MainCategories field structure
print("\n=== Sample MainCategories field from exhibitors ===")
for i, row in exhibitors_df.head(5).iterrows():
    print(f"Exhibitor: {row['Name']}")
    print(f"MainCategories: {row['MainCategories']}")
    categories = str(row['MainCategories']).split('|')
    print(f"Parsed categories: {categories}")
    print()

# Check for any data consistency issues
print("\n=== Data Consistency Checks ===")
print(f"Unique visitor IDs: {visitors_df['id'].nunique() if 'id' in visitors_df.columns else 'id column not found'}")
print(f"Unique exhibitor IDs: {exhibitors_df['exhibitorid'].nunique() if 'exhibitorid' in exhibitors_df.columns else 'exhibitorid column not found'}")
print(f"Unique question IDs: {questions_df['id'].nunique() if 'id' in questions_df.columns else questions_df.columns[0] + ': ' + str(questions_df[questions_df.columns[0]].nunique())}")
print(f"Unique answer IDs: {answers_df['id'].nunique() if 'id' in answers_df.columns else answers_df.columns[0] + ': ' + str(answers_df[answers_df.columns[0]].nunique())}")
print(f"Unique category IDs: {exhibitor_categories_df['categoryId'].nunique() if 'categoryId' in exhibitor_categories_df.columns else 'categoryId column not found'}")

# Check if all answer IDs in visitor_answers exist in the answers table
if 'answerId' in visitor_answers_enriched.columns:
    all_answer_ids = visitor_answers_enriched['answerId'].unique()
    if 'id' in answers_df.columns:
        existing_answer_ids = answers_df['id'].unique()
    elif answers_df.columns.tolist()[0]:  # Use first column if 'id' doesn't exist
        existing_answer_ids = answers_df[answers_df.columns[0]].unique()
    else:
        existing_answer_ids = []
    
    missing_answer_ids = set([str(x) for x in all_answer_ids]) - set([str(x) for x in existing_answer_ids])
    print(f"\nMissing answer IDs: {len(missing_answer_ids)}")
    if len(missing_answer_ids) > 0:
        print(f"Examples: {list(missing_answer_ids)[:5]}")

# Check if all category IDs in exhibitors exist in the categories table
if 'MainCategories' in exhibitors_df.columns:
    all_category_ids = []
    for cats in exhibitors_df['MainCategories']:
        if pd.notnull(cats):
            all_category_ids.extend(str(cats).split('|'))
    unique_category_ids = set(all_category_ids)
    
    if 'categoryId' in exhibitor_categories_df.columns:
        existing_category_ids = set(exhibitor_categories_df['categoryId'].astype(str))
        missing_category_ids = unique_category_ids - existing_category_ids
        print(f"\nMissing category IDs: {len(missing_category_ids)}")
        if len(missing_category_ids) > 0:
            print(f"Examples: {list(missing_category_ids)[:5]}")


=== Actual column names in each dataset ===
Visitors columns: ['email', 'gender', 'id', 'data']
Answers columns: ['answerId', 'answer_text', 'questionId']
Questions columns: ['questionId', 'questionTypeId', 'stepId', 'question_text']
Exhibitors columns: ['exhibitorid', 'Name', 'MainCategories']
Exhibitor Categories columns: ['categoryId', 'categoryName']

=== Analysis of Visitors ===

Shape: (100, 4)
Columns: ['email', 'gender', 'id', 'data']

Data types:
email     object
gender    object
id        object
data      object
dtype: object

First 5 rows:
                                      email gender                        id  \
0                   emilija+100_L8gA@bss.mk      F  67b70a9f2d21f543a1096602   
1   aleksandar.dimkov+mitt1_n5eA@bss.com.mk      M  67ada1ee197e604dd2722d1b   
2             daniela.p+150_YhiF@bss.com.mk      F  67b44fef197e604dd2722d3d   
3                 tanja+182_jiPa@bss.com.mk      F  67b45018197e604dd2722d3e   
4  aleksandar.dimkov+mitt10_V0iB@bss.com.m

## Parse visitor answers JSON

The visitor answers are stored as JSON in the data column... not great but gotta work with what we have

In [16]:
def extract_visitor_answers(visitor_row):
    flattened = []
    try:
        data = json.loads(visitor_row["data"])
        for ans in data:
            flattened.append({
                "visitor_id": visitor_row["id"],
                "email": visitor_row["email"],
                "gender": visitor_row["gender"],
                "questionId": ans.get("questionId"),
                "answerId": ans.get("answerId"),
                "answerValue": ans.get("answerValue"),
                "answerTypeId": ans.get("answerTypeId"),
            })
    except:
        # Just skip if there's any issues
        pass
    return flattened

# apply to all visitors and flatten
flat_answers = []
for idx, row in visitors_df.iterrows():
    flat_answers.extend(extract_visitor_answers(row))

visitor_answers_df = pd.DataFrame(flat_answers)

## Fix columns for joining
Convert IDs to strings to avoid merge errors

In [17]:
# make sure IDs are strings
visitor_answers_df["questionId"] = visitor_answers_df["questionId"].astype(str)
visitor_answers_df["answerId"] = visitor_answers_df["answerId"].astype(str)
answers_df["id"] = answers_df["id"].astype(str)
answers_df["questionId"] = answers_df["questionId"].astype(str)
questions_df["id"] = questions_df["id"].astype(str)

## Join with question/answer text 
Need to get the actual text for answers and questions to make sense of the data

In [18]:
# rename columns before merge
answers_df = answers_df.rename(columns={"id": "answerId", "answer": "answer_text"})
df_answers = visitor_answers_df.merge(answers_df, on="answerId", how="left")

# fix questionId if needed
if "questionId_x" in df_answers.columns:
    df_answers.rename(columns={"questionId_x": "questionId"}, inplace=True)

# add question text
questions_df = questions_df.rename(columns={"id": "questionId", "question": "question_text"})
visitor_answers_enriched = df_answers.merge(questions_df, on="questionId", how="left")

# select relevant columns
visitor_answers_enriched = visitor_answers_enriched[[
    "visitor_id", "email", "gender", "questionId", "question_text",
    "answerId", "answer_text", "answerValue"
]]

# Profile Analysis

Let's look at what questions & answers we have

In [19]:
# What are the common questions?
top_questions = visitor_answers_enriched["question_text"].value_counts().head()
print("Most common questions:\n", top_questions)

# What are the common answers?
answer_distribution = visitor_answers_enriched.copy()
answer_distribution["combined_answer"] = answer_distribution["answer_text"].fillna(answer_distribution["answerValue"])
top_answers = answer_distribution["combined_answer"].value_counts().head()
print("\nMost common answers:\n", top_answers)

# Take a look at answers by question
answer_by_question = answer_distribution.groupby(
    ["question_text", "answer_text"]
).size().reset_index(name="count").sort_values(["question_text", "count"], ascending=[True, False])

print("\nSample of answers per question:")
for q in top_questions.index[:3]:
    print(f"\n{q}:")
    print(answer_by_question[answer_by_question["question_text"] == q].head(3))

Most common questions:
 question_text
Reason for Attending the Event                                      100
Which of the following best describes your job function?             87
Please indicate your company's main area of business                 87
What role do you play in the purchasing decision making process?     87
What is your company's annual purchasing budget?                     30
Name: count, dtype: int64

Most common answers:
 combined_answer
                1163
True             200
Russia            78
Travel Agent      54
BSS               41
Name: count, dtype: int64

Sample of answers per question:

Reason for Attending the Event:
                    question_text                       answer_text  count
9  Reason for Attending the Event   To source products and services     35
7  Reason for Attending the Event     To obtain general information     24
8  Reason for Attending the Event  To promote products and services     20

Which of the following best describes y

## Exhibitor profile analysis
Expand exhibitor categories (they can have multiple)

In [20]:
# the categories are pipe-separated - need to split them out
exhibitors_df["MainCategories"] = exhibitors_df["MainCategories"].astype(str)
exhibitors_expanded = exhibitors_df.assign(
    categoryId=exhibitors_df["MainCategories"].str.split("|")
).explode("categoryId")

exhibitors_expanded["categoryId"] = exhibitors_expanded["categoryId"].astype(int)

# join category names
exhibitors_with_cats = exhibitors_expanded.merge(
    exhibitor_categories_df, on="categoryId", how="left"
)

# which categories are most common?
category_counts = exhibitors_with_cats["categoryName"].value_counts()
print("Top categories:\n", category_counts.head())

# which exhibitors have the most categories?
cats_per_exhibitor = (
    exhibitors_with_cats.groupby(["exhibitorid", "Name"])["categoryId"]
    .count()
    .reset_index(name="num_categories")
    .sort_values("num_categories", ascending=False)
)
print("\nExhibitors with most categories:\n", cats_per_exhibitor.head())

Top categories:
 categoryName
12.2 NTO: National tourism office    5
14.6 Gastro tourism                  5
1.3 Hostel / Motel                   5
3.2 Authorized travel agency         4
17.2 Payment service provider        4
Name: count, dtype: int64

Exhibitors with most categories:
     exhibitorid                       Name  num_categories
20        72153  Elite Adventures Holidays              12
17        55311  Sunny Adventures Journeys              11
14        48028  Prime Adventures Journeys              11
7         21865    Dream Holidays Holidays              11
19        68142       Global Tours Voyages              10


# Keyword → Category Mapping

Now lets map visitor answers to relevant exhibitor categories using fuzzy matching

In [21]:
def build_keywords_from_answers(row):
    # Build clean keyword from answer text + value
    text = str(row["answer_text"]) if pd.notnull(row["answer_text"]) else ""
    value = str(row["answerValue"]) if pd.notnull(row["answerValue"]) else ""
    kw = f"{text.strip()} {value.strip()}".strip().lower()
    # skip useless values
    if kw in {"true", "false", "nan", ""}:
        return ""
    return kw

# Get unique keywords from visitor answers
unique_keywords = visitor_answers_enriched.apply(build_keywords_from_answers, axis=1).unique()
visitor_keywords = [k for k in unique_keywords if k]
print(f"Found {len(visitor_keywords)} unique visitor keywords")

# Get category names
category_names = exhibitor_categories_df["categoryName"].unique()
print(f"Found {len(category_names)} category names")

# match keywords to categories 
keyword_to_category_map = {}
for keyword in visitor_keywords:
    # find best matching categories
    matches = process.extract(keyword, category_names, limit=3)
    
    matching_category_ids = set()
    for match_text, score in matches:
        if score > 80:  # 80% threshold
            cat_ids = exhibitor_categories_df[
                exhibitor_categories_df["categoryName"] == match_text
            ]["categoryId"].tolist()
            matching_category_ids.update(cat_ids)
    
    if matching_category_ids:
        keyword_to_category_map[keyword] = list(matching_category_ids)

# show some samples
print("\nSample keyword → category mappings:")
for i, (kw, cats) in enumerate(keyword_to_category_map.items()):
    if i >= 10: break
    print(f"{kw} → {cats}")

Found 186 unique visitor keywords
Found 96 category names

Sample keyword → category mappings:
eme → [52328, 52277]
russia → [52361]
media → [52356, 52357, 52358]
travel agent → [52337, 52341, 52286]
bregalnicka 12 → [52336, 52337, 52338]
to source products and services → [52312, 52294, 52335]
event management → [52332]
5 - 10 million rubles → [52329, 52325, 52326]
to promote products and services → [52296, 52294, 52335]
formation of tourist products → [52320, 52316, 52335]


# Visitor-Exhibitor Matching 

Build maps of visitors → categories and exhibitors → categories, then match them up

In [22]:
# Build exhibitor_id → category_ids map
exhibitor_to_categories = (
    exhibitors_with_cats
    .groupby("exhibitorid")["categoryId"]
    .apply(set)
    .to_dict()
)

# Build visitor_id → category_ids map based on their answers
visitor_to_categories = {}
for _, row in visitor_answers_enriched.iterrows():
    visitor_id = row["visitor_id"]
    keyword = build_keywords_from_answers(row)
    if keyword and keyword in keyword_to_category_map:
        category_ids = keyword_to_category_map[keyword]
        if visitor_id not in visitor_to_categories:
            visitor_to_categories[visitor_id] = set()
        visitor_to_categories[visitor_id].update(category_ids)

# Calculate match scores 
matching_results = []
for visitor_id, visitor_cats in visitor_to_categories.items():
    for exhibitor_id, exhibitor_cats in exhibitor_to_categories.items():
        common_categories = visitor_cats & exhibitor_cats
        if common_categories:
            # Simple scoring: penalize exhibitors with too many categories
            base_score = len(common_categories)
            penalty = np.log1p(len(exhibitor_cats))  
            score = base_score / penalty
            matching_results.append((visitor_id, exhibitor_id, score))

# convert to dataframe
matching_df = pd.DataFrame(matching_results, columns=["visitor_id", "exhibitor_id", "match_score"])

# add names and emails
matching_df = matching_df.merge(
    visitors_df[["id", "email"]], left_on="visitor_id", right_on="id", how="left"
).drop(columns="id")

matching_df = matching_df.merge(
    exhibitors_df[["exhibitorid", "Name"]], left_on="exhibitor_id", right_on="exhibitorid", how="left"
).drop(columns="exhibitorid")

# sort scores
matching_df = matching_df[["email", "Name", "match_score"]].sort_values(
    ["email", "match_score"], ascending=[True, False]
)

print(f"Generated {len(matching_df)} matches")
print("\nSample matches:")
print(matching_df.head(10))

Generated 1194 matches

Sample matches:
                     email                       Name  match_score
89  3990147_SeNs@gmail.com             Turkey Travels     1.442695
94  3990147_SeNs@gmail.com         Exotic Tours Tours     1.116221
91  3990147_SeNs@gmail.com      Indian Travel Company     0.721348
86  3990147_SeNs@gmail.com  Royal Holidays Adventures     0.621335
92  3990147_SeNs@gmail.com      Exotic Europe Travels     0.621335
90  3990147_SeNs@gmail.com     Russian Travel Company     0.513898
93  3990147_SeNs@gmail.com          Dream Tours Tours     0.434294
87  3990147_SeNs@gmail.com       Global Tours Voyages     0.417032
85  3990147_SeNs@gmail.com    Dream Holidays Holidays     0.402430
88  3990147_SeNs@gmail.com  Elite Adventures Holidays     0.389871


# Generate top-7 recommendations

Both for visitors → exhibitors and exhibitors → visitors

In [23]:
# Top 7 exhibitors for each visitor
top_exhibitors_by_visitor = (
    matching_df.groupby("email")
    .apply(lambda x: x.nlargest(7, "match_score"))
    .reset_index(drop=True)
)

print("Top exhibitor recommendations per visitor:")
print(top_exhibitors_by_visitor.head(15))

# Top 7 visitors for each exhibitor 
top_visitors_by_exhibitor = (
    matching_df.groupby("Name")
    .apply(lambda x: x.nlargest(7, "match_score"))
    .reset_index(drop=True)
)

print("\nTop visitor recommendations per exhibitor:")
print(top_visitors_by_exhibitor.head(15))

Top exhibitor recommendations per visitor:
                          email                       Name  match_score
0        3990147_SeNs@gmail.com             Turkey Travels     1.442695
1        3990147_SeNs@gmail.com         Exotic Tours Tours     1.116221
2        3990147_SeNs@gmail.com      Indian Travel Company     0.721348
3        3990147_SeNs@gmail.com  Royal Holidays Adventures     0.621335
4        3990147_SeNs@gmail.com      Exotic Europe Travels     0.621335
5        3990147_SeNs@gmail.com     Russian Travel Company     0.513898
6        3990147_SeNs@gmail.com          Dream Tours Tours     0.434294
7   3990147_SeNs_09Hr@gmail.com             Turkey Travels     1.442695
8   3990147_SeNs_09Hr@gmail.com         Exotic Tours Tours     1.116221
9   3990147_SeNs_09Hr@gmail.com      Indian Travel Company     0.721348
10  3990147_SeNs_09Hr@gmail.com  Royal Holidays Adventures     0.621335
11  3990147_SeNs_09Hr@gmail.com      Exotic Europe Travels     0.621335
12  3990147_SeNs_09Hr

/var/folders/fl/374pqpm17q5fl8r9c1vws6wh0000gn/T/ipykernel_87884/2138214895.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(7, "match_score"))
/var/folders/fl/374pqpm17q5fl8r9c1vws6wh0000gn/T/ipykernel_87884/2138214895.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.nlargest(7, "match_score"))


# Testing

A few basic tests to make sure everything's working as expected

In [24]:
import unittest

class TestMatchingSystem(unittest.TestCase):
    
    def test_flattening_function(self):
        test_row = {
            "id": "abc123",
            "email": "test@example.com",
            "gender": "M",
            "data": '[{"questionId": "q1", "answerId": "a1", "answerValue": "Test Value"}]'
        }
        result = extract_visitor_answers(test_row)
        self.assertEqual(len(result), 1)
        self.assertEqual(result[0]["visitor_id"], "abc123")
        self.assertEqual(result[0]["answerValue"], "Test Value")

    def test_keyword_mapping_exists(self):
        # Make sure we have mapped some keywords
        self.assertGreater(len(keyword_to_category_map), 0)
        
    def test_scoring_function(self):
        # Test that penalty works as expected 
        score1 = 3 / np.log1p(10)
        score2 = 3 / np.log1p(20)
        self.assertGreater(score1, score2)  # fewer categories = higher score

    def test_dataframe_structure(self):
        # Check output has the right columns
        self.assertIn("email", matching_df.columns)
        self.assertIn("Name", matching_df.columns) 
        self.assertIn("match_score", matching_df.columns)

# Run the tests
if __name__ == "__main__":
    print("Running tests...")
    unittest.main(argv=[''], exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.001s

OK


Running tests...
